# Visualização da Informação
## Escola de Matemática Aplicada - Fundação Getúlio Vargas
## Mestrado em Modelagem Matemática

Aluno: Gianlucca Devigili<br>
Github do projeto: https://github.com/GDevigili/information-visualization-homeworks

## Trabalho 2: Análise e reprodução de uma visualização reconhecida ou relevante historicamente

+ Parte1: Encontrar os dados (compartilhar referências de dados no slack)
+ Parte2: Fazer uma análise de qual seria a função pretendida com a visualização proposta. 
+ Parte3: Fazer uma reprodução da visualização escolhida utilizando uma ferramenta computacional atual (de preferência a mesma escolhida por vocês no trabalho 1)
+ Parte4: Propor alguma modificação (fundamentando conceitualmente) na visualização proposta. Exemplo: Incluir anotação, incluir interatividade, modificar título ou legenda, adicionar informação, etc.

In [1]:
import pandas as pd
import altair as alt
import plotly.express as px

# scrapping
from bs4 import BeautifulSoup as bs
import requests
import vega_datasets

A visualização escolhida foi a de **Florence Nightingale** referente às __causas de morte na guerra da Crimeia__ (1853-1856)


![](https://wp-assets.highcharts.com/www-highcharts-com/blog/wp-content/uploads/2015/05/30154347/Nightingale%E2%80%99s-diagram-1.jpg)

## (1) Aquisição dos dados

Tive certa dificuldade de encontrar o dataframe em `.csv`, portanto fiz um scrapping simples da tabela encontrada em <https://medium.com/@zumaia/how-florence-nightingales-diagram-saved-millions-of-people-from-fatal-epidemic-disease-dfad7348b05d>.

In [2]:
html = requests.get("http://understandinguncertainty.org/node/214.")
soup = bs(html.text)

# seleciona a tag table
table = soup.find('table')

# extrai o nome das colunas da tabela
column_names = []
for name in table.find_all('tr')[1]:
    if name != "\n":
        column_names.append(name.text)

for i in range(2, 5):
    column_names[i] += '(Deaths)'
    column_names[i+3] += '(Annual rate per 1000)'
        
# extrai as linhas da tabela
trows = soup.find('tbody').find_all('tr')[2:]
tcols = []

# itera sobre as linhas
for tr in soup.find('tbody').find_all('tr')[2:]:
    # pega os valores da lista de 'td' e remove ' ' e '\n'
    cols = [val.text.replace('\n', '').replace(' ', '').replace('_', '') for val in tr.find_all('td')]
    # adiciona a lista de colunas
    tcols.append(cols)

# cria o dataset 
column_names[0] = 'Date'
df_crimea = pd.DataFrame(tcols, columns = column_names)

'''
# cria um dataset para as datas
dict_months = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
month_year = []

for date in df_crimea['Date']:
    # cria uma lista de listas do tipo [[mes, ano], [mes, ano], [mes, ano], ...]
    month_year.append([int(dict_months[date[0:3]]), int(date[3:])])
    
df_date = pd.DataFrame(month_year, columns=['Month', 'Year'])
'''
# converte para int64 e float64 as colunas numéricas
for col in df_crimea.columns[1:]:
    df_crimea[col] = df_crimea[col].apply(pd.to_numeric)
    
# df_crimea['Date'] = vega_datasets.data.crimea()['date']

df_crimea

,Date,Average size of army,Zymotic diseases(Deaths),Wounds & injuries(Deaths),All other causes(Deaths),Zymotic diseases(Annual rate per 1000),Wounds & injuries(Annual rate per 1000),All other causes(Annual rate per 1000)
0,Apr1854,8571,1,0,5,1.4,0.0,7.0
1,May1854,23333,12,0,9,6.2,0.0,4.6
2,Jun1854,28333,11,0,6,4.7,0.0,2.5
3,Jul1854,28722,359,0,23,150.0,0.0,9.6
4,Aug1854,30246,828,1,30,328.5,0.4,11.9
5,Sep1854,30290,788,81,70,312.2,32.1,27.7
6,Oct1854,30643,503,132,128,197.0,51.7,50.1
7,Nov1854,29736,844,287,106,340.6,115.8,42.8
8,Dec1854,32779,1725,114,131,631.5,41.7,48.0
9,Jan1855,32393,2761,83,324,1022.8,30.7,120.0


In [3]:
df_crimea_melted = pd.melt(
                    df_crimea, 
                    id_vars = ['Date'], 
                    value_vars = ['Zymotic diseases(Deaths)', 'Wounds  & injuries(Deaths)', 'All other causes(Deaths)'],
                    var_name = 'Cause', value_name = 'Deaths'
                    )
df_crimea_melted.head()

,Date,Cause,Deaths
0,Apr1854,Zymotic diseases(Deaths),1
1,May1854,Zymotic diseases(Deaths),12
2,Jun1854,Zymotic diseases(Deaths),11
3,Jul1854,Zymotic diseases(Deaths),359
4,Aug1854,Zymotic diseases(Deaths),828


## (2) Análise da Função da Visualização


## (3) Reprodução da Visualização

Primeiro vamos separar os dados por gráfico, sendo o primeiro indo de abril de 1854 até março de 1855 e o segundo de abril de 1855 até março de 1856.
Depois usamos o método `melt` do pandas para transformar o dataset de modo que ele tenha 3 colunas: `Date`, `Death` e `Cause`

In [4]:
df1 = pd.melt(
        df_crimea[:12], 
        id_vars = ['Date'], 
        value_vars = ['Zymotic diseases(Deaths)', 'Wounds  & injuries(Deaths)', 'All other causes(Deaths)'],
        var_name = 'Cause', value_name = 'Deaths'
        )
df2 = pd.melt(
        df_crimea[12:], 
        id_vars = ['Date'], 
        value_vars = ['Wounds  & injuries(Deaths)', 'All other causes(Deaths)', 'Zymotic diseases(Deaths)'],
        var_name = 'Cause', value_name = 'Deaths'
        )
df1.head()

,Date,Cause,Deaths
0,Apr1854,Zymotic diseases(Deaths),1
1,May1854,Zymotic diseases(Deaths),12
2,Jun1854,Zymotic diseases(Deaths),11
3,Jul1854,Zymotic diseases(Deaths),359
4,Aug1854,Zymotic diseases(Deaths),828


In [16]:
colors = ['#b0c3d1', '#4c3933', '#fac0bd']

fig1 = px.bar_polar(
    df1, 
    r = 'Deaths',
    color = 'Cause', 
    theta = 'Date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig1.show()

In [17]:
colors.reverse

fig2 = px.bar_polar(
    df2, 
    r = 'Deaths',
    color = 'Cause', 
    theta = 'Date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig2.show()


## (4) Modificações Propostas